# Imports and Keys

In [15]:
import openai
from openai import OpenAI
import pandas as pd
import numpy as np
import pickle
import requests
import re
from datetime import datetime
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

def create_minute_gap(seconds):
    print("creating minute gap to avoid TPM limit error...") 
    counter = 0
    while counter < (1000000000 * (seconds/60)):
        counter += 1

In [16]:
API=""

In [17]:
agent_key=""

In [18]:
client = OpenAI(api_key=API)

# Experiment Procedure

Procedure:
1. Remind agents with the setting of the experiment
2. instruct them to decide the allocation to private and public account
3. aggregate the decision and compute the rewards
    - private internal return (NFT)
    - public internal return (public goods from own contribution)
    - external return (public goods from others\' contribution)
4. give feedback
    - total contrubution made this round
    - total reward
    - breakdown of total reward
    - situation information
        - DAO crashed?
        - NF succeeded?
    - historical contribution level
5. insruct them to make decision given feedback and reminder of demographics
6. repeat 3~5 for configured rounds

Prompts:
- Assistent Configuration Prompt
You are a participant of the experiment. You are asked to allocate your endowment to two different accounts: private account and public account. The allocation to the private account will provide certain return only to you, but the allocation to the public account will equally provide everyone certain return. You are not allowed to reuse the return earned in previous round, and fixed amount of endowment will be provided to you every round. You will be informed with your demographic information and the detailed settings of the experiment at the beginning of the experiment, so you must imitate human decisions given those information. You should explain your thought process and reasons of your decisions in detail.
- Initial Instruction Prompt
    - Control
You are {agent.age} years old {agent.gender} who is {agent.risk_preference} to some extent. You are participating in the experiment where you are asked to allocate your endowment to two different accounts: private account and public account. The allocation to the private account will provide return only to you, but the allocation to the public account will provide everyone certain return. You are not allowed to reuse the return earned in previous round, and fixed endowment of 100 will be provided to you every round. {self.configs['num_agents'] - 1} other people are participating in this experiment, facing same decision choices as you. Your return will be computed by: private_account + public_account * {self.configs['r_NFT'] + self.configs['r_public']} + sum_of_others\'_public account * {self.configs['h_public']}. Now, make your first decision on the allocation of 100 endowment to private and public account. You should explain your thought process and reasons of your decisions in detail.
    - Treatment1
You are {agent.age} years old {agent.gender} who is {agent.risk_preference} to some extent. You are participating in the experiment where you are asked to allocate your endowment to two different accounts: private account and public account. The allocation to the private account will provide return only to you, but the allocation to the public account will provide everyone certain return. The contributions to the public account will be managed by Decentralized Autonomous Organization (DAO), which has a certain probability of crashing, generating no public benefits to anyone from the public account. The return you will be getting from the allocation to public account will also includes a return from Non-Fungible-Token (NFT) (only paid by chance) you will be obtaining when you allocate your endowment to public account. You are not allowed to reuse the return earned in previous round, and fixed endowment of 100 will be provided to you every round. {self.configs['num_agents'] - 1} other people are participating in this experiment, facing same decision choices as you. Your return will be computed by: private_account + public_account * r_NFT + public_account * r_public + sum_of_others\'_public account * h_public. r_NFT is {self.configs['r_NFT']} with probability of {self.configs['p(s_NFT=1)']} and 0 with {1 - self.configs['p(s_NFT=1)']}. r_public is {self.configs['r_public']} with probability of {self.configs['p(s_DAO=1)']} and 0 with {1 - self.configs['p(s_DAO=1)']}. h_public is {self.configs['h_public']} with probability of {self.configs['p(s_DAO=1)']} and 0 with {1 - self.configs['p(s_DAO=1)']}, which is linked with r_public. So if r_public is {self.configs['r_public']} then h_public is {self.configs['h_public']}, and if one of them is 0 then the other one is also 0, because they are decided together. Now, make your first decision on the allocation of 100 endowment to private and public account. You should explain your thought process and reasons of your decisions in detail, and report the amount allocated to private and public account.
    - Treatment 2
You are {agent.age} years old {agent.gender} who is {agent.risk_preference} to some extent. You are participating in the experiment where you are asked to allocate your endowment to two different accounts: private account and public account. The allocation to the private account will provide return only to you, but the allocation to the public account will provide everyone certain return. The contributions to the public account will be managed by Decentralized Autonomous Organization (DAO), which has a certain probability of crashing, generating no public benefits to anyone from the public account. The return you will be getting from the allocation to public account will also includes a return from Non-Fungible-Token (NFT) (only paid by chance) you will be obtaining when you allocate your endowment to public account. You are not allowed to reuse the return earned in previous round, and fixed endowment of 100 will be provided to you every round. {self.configs['num_agents'] - 1} other people are participating in this experiment, facing same decision choices as you. Your return will be computed by: private_account + public_account * r_NFT + public_account * r_public + sum_of_others\'_public account * h_public. r_NFT is {self.configs['r_NFT']} with probability of {self.configs['p(s_NFT=1)']} and 0 with {1 - self.configs['p(s_NFT=1)']}. r_public is {self.configs['r_public']} with probability of {self.configs['p(s_DAO=1)']} and 0 with {1 - self.configs['p(s_DAO=1)']}. h_public is {self.configs['h_public']} with probability of {self.configs['p(s_DAO=1)']} and 0 with {1 - self.configs['p(s_DAO=1)']}, which is linked with r_public. So if r_public is {self.configs['r_public']} then h_public is {self.configs['h_public']}, and if one of them is 0 then the other one is also 0, because they are decided together. Importantly, if r_public and h_public are 0, then it causes r_NFT to be 0. However, r_NFT being 0 does not necessarily cause r_public or h_public to be 0. Now, make your first decision on the allocation of 100 endowment to private and public account. You should explain your thought process and reasons of your decisions in detail and report the amount allocated to private and public account.
    - Treatment3
You are {agent.age} years old {agent.gender} who is {agent.risk_preference} to some extent. You are participating in the experiment where you are asked to allocate your endowment to two different accounts: private account and public account. The allocation to the private account will provide return only to you, but the allocation to the public account will provide everyone certain return. The contributions to the public account will be managed by Decentralized Autonomous Organization (DAO), which has a certain probability of crashing, generating no public benefits to anyone from the public account. The return you will be getting from the allocation to public account will also includes a return from Non-Fungible-Token (NFT) (only paid by chance) you will be obtaining when you allocate your endowment to public account. You are not allowed to reuse the return earned in previous round, and fixed endowment of 100 will be provided to you every round. {self.configs['num_agents'] - 1} other people are participating in this experiment, facing same decision choices as you. Your return will be computed by: private_account + public_account * r_NFT + public_account * r_public + sum_of_others\'_public account * h_public. r_NFT is {self.configs['r_NFT']} with probability of {self.configs['p(s_NFT=1)']} and 0 with {1 - self.configs['p(s_NFT=1)']}. r_public is {self.configs['r_public']} with probability of {self.configs['p(s_DAO=1)']} and 0 with {1 - self.configs['p(s_DAO=1)']}. h_public is {self.configs['h_public']} with probability of {self.configs['p(s_DAO=1)']} and 0 with {1 - self.configs['p(s_DAO=1)']}, which is linked with r_public. So if r_public is {self.configs['r_public']} then h_public is {self.configs['h_public']}, and if one of them is 0 then the other one is also 0, because they are decided together. Importantly, if r_public and h_public are 0, then it causes r_NFT to be 0. However, r_NFT being 0 does not necessarily cause r_public or h_public to be 0. Another important point is that if one of r_NFT or (r_public and h_public) is positive, and when the other one is 0, then the value of the other one is redrawn using the same probability. This means that if one of the r_NFT or (r_public and h_public) is positive, then it increases the chance of the other to be positive. However if (r_public and h_public) were 0 even after redraw, r_NFT will become 0. Now, make your first decision on the allocation of 100 endowment to private and public account. You should explain your thought process and reasons of your decisions in detail and report the amount allocated to private and public account.
- Feedback & Next Instruction Prompt
    - Control
After the allocation, you received {payoff}. Total of {total_contributions} were allocated to public accounts by all participants, where other participants contributed {total_contributions - public}. Historically, the total contributions to the public account is {self.total_contributions_record}, where each of other participants have chosen to allocate {agent.others_average_publics} (average level of other individual\'s contribution) historically. {situation_messages['DAO']} {situation_messages['NFT']} Now, given this information, your demographics, and experiment settings, make your next decision on the allocation of 100 endowment to private and public account. You should explain your thought process and reasons of your decisions in detail and report the amount allocated to private and public account.
    - Treatment1
After the allocation, you received {payoff}. Total of {total_contributions} were allocated to public accounts by all participants, where other participants contributed {total_contributions - public}. Historically, the total contributions to the public account is {self.total_contributions_record}, where each of other participants have chosen to allocate {agent.others_average_publics} (average level of other individual\'s contribution) historically. {situation_messages['DAO']} {situation_messages['NFT']} Now, given this information, your demographics, and experiment settings, make your next decision on the allocation of 100 endowment to private and public account. You should explain your thought process and reasons of your decisions in detail and report the amount allocated to private and public account.
    - Treatment 2
After the allocation, you received {payoff}. Total of {total_contributions} were allocated to public accounts by all participants, where other participants contributed {total_contributions - public}. Historically, the total contributions to the public account is {self.total_contributions_record}, where each of other participants have chosen to allocate {agent.others_average_publics} (average level of other individual\'s contribution) historically. {situation_messages['DAO']} {situation_messages['NFT']} Now, given this information, your demographics, and experiment settings, make your next decision on the allocation of 100 endowment to private and public account. You should explain your thought process and reasons of your decisions in detail and report the amount allocated to private and public account.
    - Treatment3
After the allocation, you received {payoff}. Total of {total_contributions} were allocated to public accounts by all participants, where other participants contributed {total_contributions - public}. Historically, the total contributions to the public account is {self.total_contributions_record}, where each of other participants have chosen to allocate {agent.others_average_publics} (average level of other individual\'s contribution) historically. {situation_messages['DAO']} {situation_messages['NFT']} Now, given this information, your demographics, and experiment settings, make your next decision on the allocation of 100 endowment to private and public account. You should explain your thought process and reasons of your decisions in detail and report the amount allocated to private and public account.

# Configurations

In [19]:
def treatment1(p_sDAO, p_sNFT):
    sDAO = np.random.choice([0, 1], p=[1-p_sDAO, p_sDAO])
    sNFT = np.random.choice([0, 1], p=[1-p_sNFT, p_sNFT])
    
    situation = {
        "DAO": "DAO is functioning, so contributions were effectively converted to public goods, and everyone enjoyed a share of it." if sDAO==1 else "DAO crashed, so the contributions were not converted to public goods, and nobody enjoyed the benefit of public goods.",
        "NFT": "You succeeded in obtaining a private return on contribution from through NFT." if sNFT==1 else "You failed to recieve a private return on contribution through NFT."
    }
    return situation, sDAO, sNFT

def treatment2(p_sDAO, p_sNFT):
    sDAO = np.random.choice([0, 1], p=[1-p_sDAO, p_sDAO])
    sNFT = np.random.choice([0, 1], p=[1-p_sNFT, p_sNFT])
    
    if sDAO==0:
        sNFT = 0
    
    situation = {
        "DAO": "DAO is functioning, so contributions were effectively converted to public goods, and everyone enjoyed a share of it." if sDAO==1 else "DAO crashed, so the contributions were not converted to public goods, and nobody enjoyed the benefit of public goods.",
        "NFT": "You succeeded in obtaining a private return on contribution from through NFT." if sNFT==1 else "You failed to recieve a private return on contribution through NFT."
    }
    return situation, sDAO, sNFT

def treatment3(p_sDAO, p_sNFT):
    sDAO = np.random.choice([0, 1], p=[1-p_sDAO, p_sDAO])
    sNFT = np.random.choice([0, 1], p=[1-p_sNFT, p_sNFT])
    
    if (sDAO==1) & (sNFT==0):
        sNFT = np.random.choice([0, 1], p=[1-p_sNFT, p_sNFT])
    
    if (sNFT==1) & (sDAO==0):
        sDAO = np.random.choice([0, 1], p=[1-p_sDAO, p_sDAO])
    
    if sDAO==0:
        sNFT = 0 
    
    situation = {
        "DAO": "DAO is functioning, so contributions were effectively converted to public goods, and everyone enjoyed a share of it." if sDAO==1 else "DAO crashed, so the contributions were not converted to public goods, and nobody enjoyed the benefit of public goods.",
        "NFT": "You succeeded in obtaining a private return on contribution from through NFT." if sNFT==1 else "You failed to recieve a private return on contribution through NFT."
    }
    return situation, sDAO, sNFT


In [20]:
# control no risk
config_C = {
    "rounds": 5,
    "num_agents": 4,
    "endowment": 100,
    "r_NFT": 0.25,
    "r_public": 0.25, # E[r_NFT + r_public] = 0.5
    "h_public": 0.5, # E[h_public] = 0.5
    "p(s_DAO=1)": 1,
    "p(s_NFT=1)": 1,
    "group": None
}

# treatment1: independent risk
config_T1 = {
    "rounds": 5,
    "num_agents": 4,
    "endowment": 100,
    "r_NFT": 1/3,
    "r_public": 1/3, # E[r_NFT + r_public] = (r_public + r_NFT) * p(s_DAO=1) * p(s_NFT=1) + r_public * p(s_DAO=1) + r_NFT * p(s_NFT=1) = (1/3 + 1/3) * 0.25 + 1/3 * 0.5 + 1/3 * 0.5 = 0.5
    "h_public": 1, # E[h_public] = p(s_DAO=1) * h_public = 0.5
    "p(s_DAO=1)": 0.5,
    "p(s_NFT=1)": 0.5,
    "group": treatment1
}

# treatment2: 
config_T2 = {
    "rounds": 5,
    "num_agents": 4,
    "endowment": 100,
    "r_NFT": 0.5,
    "r_public": 0.5, # E[r_NFT + r_public] = (r_public + r_NFT) * p(s_DAO=1) * p(s_NFT=1) + r_public * p(s_DAO=1) = (0.5 + 0.5) * 0.25 + 0.5 * 0.5 = 0.5
    "h_public": 1, # E[h_public] = p(s_DAO=1) * h_public = 0.5
    "p(s_DAO=1)": 0.5,
    "p(s_NFT=1)": 0.5,
    "group": treatment2
}

config_T3 = {
    "rounds": 5,
    "num_agents": 4,
    "endowment": 100,
    "r_NFT": 0.5,
    "r_public": 0.4, # E[r_NFT + r_public] = (r_public + r_NFT) * p(s_DAO=1) * p(s_NFT=1) + (r_public + r_NFT) * p(s_DAO=1) * (1 - p(s_NFT=1)) * p(s_NFT=1) + r_public * p(s_DAO=1) * (1 - p(s_NFT=1)) * (1 - p(s_NFT=1)) + (r_public + r_NFT) * p(s_NFT=1) * (1 - p(s_DAO=1)) * p(s_DAO=1) = (0.4 + 0.5) * 0.5 * 0.5 + (0.4 + 0.5) * 0.5 * 0.5 * 0.5 + + 0.4 * 0.5 * 0.5 * 0.5 + (0.4 + 0.5) * 0.5 * 0.5 * 0.5 = 0.5
    "h_public": 0.8, # E[h_public] = h_public * p(s_DAO=1) + h_public * (1 - p(s_DAO=1)) * p(s_NFT=1) * p(s_DAO=1) = 0.8 * 0.5 + 0.8 * 0.5 * 0.5 * 0.5 = 0.5
    "p(s_DAO=1)": 0.5,
    "p(s_NFT=1)": 0.5,
    "group": treatment3
}

In [21]:
demographics = {
    "risk_preference": ["risk_neutral", "risk_averse", "risk_loving"],
    "risk_preference_dist": [0.1, 0.15, 0.75],
    "gender": ["male", "female"],
    "gender_dist": [0.8, 0.2], 
    "age": [20, 30, 40, 50, 60],
    "age_dist": [0.2, 0.4, 0.2, 0.1, 0.1]
}

# Library

In [22]:
class Agent():
    def __init__(self, client, agent_key, demog):
        self.client = client
        self.agent_key = agent_key
        
        self.risk_preference = demog["risk_preference"]
        self.gender = demog["gender"]
        self.age = demog["age"]
        
        self.privates = []
        self.publics = []
        self.others_average_publics = []
        
        self.decisions = []
        
    def join_experiment(self):
        self.thread = self.client.beta.threads.create()
        
    def receive_message(self):
        self.message = self.client.beta.threads.messages.create(
            thread_id=self.thread.id,
            role="user",
            content=self.content
        )
    
    def make_decision(self, print_decision=True):
        # make decision based on provided instruction or feedback
        with self.client.beta.threads.runs.stream(
            thread_id=self.thread.id,
            assistant_id=self.agent_key
        ) as self.stream:
            self.stream.until_done()
            
        # update conversation record
        self.conversation = self.client.beta.threads.messages.list(
            thread_id=self.thread.id,
            order="asc"
        )
        
        # retrieve the round decision
        self.decision = self.conversation.data[-1].content[0].text.value
        
        self.decisions.append(self.decision)
        
        if print_decision:
            print(self.decision)
    
    def leave_experiment(self):
        self.client.beta.threads.delete(thread_id=self.thread.id)
    
    

In [117]:
class Experiment():
    def __init__(self, client, agent_key, configs, demogs):
        self.client = client
        self.agent_key = agent_key
        
        self.configs = configs
        self.demogs = demogs
        
        self.agents = [Agent(self.client, self.agent_key, generate_demographics(self.demogs)) for i in range(self.configs["num_agents"])]
        
        self.total_contributions_record = []
        self.sDAOs = []
        self.sNFTs = []
        
        self.round = 1
    
    def start(self):
        for i in range(len(self.agents)):
            self.agents[i].join_experiment()
            print(f"agent {i} joined the experiment")
            
    def decision_round(self, print_decision=True):
        for i in range(len(self.agents)):
            print(f"Agent {i} round {self.round} decision")
            self.agents[i].make_decision(print_decision)
            print("\n")
        self.round += 1
    
    def end(self):
        for i in range(len(self.agents)):
            self.agents[i].leave_experiment()
            print(f"agent {i} left the experiment")
    
    def give_instruction(self):
        pass
                
    def give_feedback(self):
        pass
    
class Control(Experiment):
    def give_instruction(self):
        for i in range(len(self.agents)):
            agent = self.agents[i]
            agent.content = f"You are {agent.age} years old {agent.gender} who is {agent.risk_preference} to some extent. You are participating in the experiment where you are asked to allocate your endowment to two different accounts: private account and public account. The allocation to the private account will provide return only to you, but the allocation to the public account will provide everyone certain return. You are not allowed to reuse the return earned in previous round, and fixed endowment of 100 will be provided to you every round. {self.configs['num_agents'] - 1} other people are participating in this experiment, facing same decision choices as you. Your return will be computed by: private_account + public_account * {self.configs['r_NFT'] + self.configs['r_public']} + sum_of_others\'_public account * {self.configs['h_public']}. Now, make your first decision on the allocation of 100 endowment to private and public account. You should explain your thought process and reasons of your decisions in detail."
            agent.receive_message()
            print(f"agent {i} received instruction")
                
    def give_feedback(self, privates, publics):
        total_contributions = sum(publics)
        
        self.total_contributions_record.append(total_contributions)
        
        if self.configs["group"]:
            situation_messages, sDAO, sNFT = self.configs["group"](self.configs["p(s_DAO=1)"], self.configs["p(s_NFT=1)"])
        else: 
            situation_messages, sDAO, sNFT = None, 1, 1
        self.sDAOs.append(sDAO)
        self.sNFTs.append(sNFT)
            
        for i in range(len(self.agents)):
            agent = self.agents[i]
            private = privates[i]
            agent.privates.append(private)
            public = publics[i]
            agent.publics.append(public)
            others_average_public = sum([publics[j] for j in range(len(publics)) if j!=i]) / (self.configs["num_agents"] - 1)
            agent.others_average_publics.append(others_average_public)
            payoff = private + public * (self.configs["r_NFT"] * sNFT + self.configs["r_public"] * sDAO) + (total_contributions - public) * self.configs["h_public"] * sDAO
        
            if situation_messages:
                agent.content = f"After the allocation, you received {payoff}. Total of {total_contributions} were allocated to public accounts by all participants, where other participants contributed {total_contributions - public}. Historically, the total contributions to the public account is {self.total_contributions_record}, where each of other participants have chosen to allocate {agent.others_average_publics} (average level of other individual\'s contribution) historically. {situation_messages['DAO']} {situation_messages['NFT']} Now, given this information, your demographics, and experiment settings, make your next decision on the allocation of 100 endowment to private and public account. You should explain your thought process and reasons of your decisions in detail and report the amount allocated to private and public account."
                agent.receive_message()
            else: 
                agent.content = f"After the allocation, you received {payoff}. Total of {total_contributions} were allocated to public accounts by all participants, where other participants contributed {total_contributions - public}. Historically, the total contributions to the public account is {self.total_contributions_record}, where each of other participants have chosen to allocate {agent.others_average_publics} (average level of other individual\'s contribution) historically. Now, given this information, your demographics, and experiment settings, make your next decision on the allocation of 100 endowment to private and public account. You should explain your thought process and reasons of your decisions in detail and report the amount allocated to private and public account."
                agent.receive_message()
            print(f"agent {i} received feedback")
    
class Treatment1(Experiment):
    def give_instruction(self):
        for i in range(len(self.agents)):
            agent = self.agents[i]
            agent.content = f"You are {agent.age} years old {agent.gender} who is {agent.risk_preference} to some extent. You are participating in the experiment where you are asked to allocate your endowment to two different accounts: private account and public account. The allocation to the private account will provide return only to you, but the allocation to the public account will provide everyone certain return. The contributions to the public account will be managed by Decentralized Autonomous Organization (DAO), which has a certain probability of crashing, generating no public benefits to anyone from the public account. The return you will be getting from the allocation to public account will also includes a return from Non-Fungible-Token (NFT) (only paid by chance) you will be obtaining when you allocate your endowment to public account. You are not allowed to reuse the return earned in previous round, and fixed endowment of 100 will be provided to you every round. {self.configs['num_agents'] - 1} other people are participating in this experiment, facing same decision choices as you. Your return will be computed by: private_account + public_account * r_NFT + public_account * r_public + sum_of_others\'_public account * h_public. r_NFT is {self.configs['r_NFT']} with probability of {self.configs['p(s_NFT=1)']} and 0 with {1 - self.configs['p(s_NFT=1)']}. r_public is {self.configs['r_public']} with probability of {self.configs['p(s_DAO=1)']} and 0 with {1 - self.configs['p(s_DAO=1)']}. h_public is {self.configs['h_public']} with probability of {self.configs['p(s_DAO=1)']} and 0 with {1 - self.configs['p(s_DAO=1)']}, which is linked with r_public. So if r_public is {self.configs['r_public']} then h_public is {self.configs['h_public']}, and if one of them is 0 then the other one is also 0, because they are decided together. Now, make your first decision on the allocation of 100 endowment to private and public account. You should explain your thought process and reasons of your decisions in detail, and report the amount allocated to private and public account."
            agent.receive_message()
            print(f"agent {i} received instruction")
                
    def give_feedback(self, privates, publics):
        total_contributions = sum(publics)
        
        self.total_contributions_record.append(total_contributions)
        
        if self.configs["group"]:
            situation_messages, sDAO, sNFT = self.configs["group"](self.configs["p(s_DAO=1)"], self.configs["p(s_NFT=1)"])
        else: 
            situation_messages, sDAO, sNFT = None, 1, 1
        self.sDAOs.append(sDAO)
        self.sNFTs.append(sNFT)
            
        for i in range(len(self.agents)):
            agent = self.agents[i]
            private = privates[i]
            agent.privates.append(private)
            public = publics[i]
            agent.publics.append(public)
            others_average_public = sum([publics[j] for j in range(len(publics)) if j!=i]) / (self.configs["num_agents"] - 1)
            agent.others_average_publics.append(others_average_public)
            payoff = private + public * (self.configs["r_NFT"] * sNFT + self.configs["r_public"] * sDAO) + (total_contributions - public) * self.configs["h_public"] * sDAO
        
            if situation_messages:
                agent.content = f"After the allocation, you received {payoff}. Total of {total_contributions} were allocated to public accounts by all participants, where other participants contributed {total_contributions - public}. Historically, the total contributions to the public account is {self.total_contributions_record}, where each of other participants have chosen to allocate {agent.others_average_publics} (average level of other individual\'s contribution) historically. {situation_messages['DAO']} {situation_messages['NFT']} Now, given this information, your demographics, and experiment settings, make your next decision on the allocation of 100 endowment to private and public account. You should explain your thought process and reasons of your decisions in detail and report the amount allocated to private and public account."
                agent.receive_message()
            else: 
                agent.content = f"After the allocation, you received {payoff}. Total of {total_contributions} were allocated to public accounts by all participants, where other participants contributed {total_contributions - public}. Historically, the total contributions to the public account is {self.total_contributions_record}, where each of other participants have chosen to allocate {agent.others_average_publics} (average level of other individual\'s contribution) historically. Now, given this information, your demographics, and experiment settings, make your next decision on the allocation of 100 endowment to private and public account. You should explain your thought process and reasons of your decisions in detail and report the amount allocated to private and public account."
                agent.receive_message()
            print(f"agent {i} received feedback")

class Treatment2(Experiment):
    def give_instruction(self):
        for i in range(len(self.agents)):
            agent = self.agents[i]
            agent.content = f"You are {agent.age} years old {agent.gender} who is {agent.risk_preference} to some extent. You are participating in the experiment where you are asked to allocate your endowment to two different accounts: private account and public account. The allocation to the private account will provide return only to you, but the allocation to the public account will provide everyone certain return. The contributions to the public account will be managed by Decentralized Autonomous Organization (DAO), which has a certain probability of crashing, generating no public benefits to anyone from the public account. The return you will be getting from the allocation to public account will also includes a return from Non-Fungible-Token (NFT) (only paid by chance) you will be obtaining when you allocate your endowment to public account. You are not allowed to reuse the return earned in previous round, and fixed endowment of 100 will be provided to you every round. {self.configs['num_agents'] - 1} other people are participating in this experiment, facing same decision choices as you. Your return will be computed by: private_account + public_account * r_NFT + public_account * r_public + sum_of_others\'_public account * h_public. r_NFT is {self.configs['r_NFT']} with probability of {self.configs['p(s_NFT=1)']} and 0 with {1 - self.configs['p(s_NFT=1)']}. r_public is {self.configs['r_public']} with probability of {self.configs['p(s_DAO=1)']} and 0 with {1 - self.configs['p(s_DAO=1)']}. h_public is {self.configs['h_public']} with probability of {self.configs['p(s_DAO=1)']} and 0 with {1 - self.configs['p(s_DAO=1)']}, which is linked with r_public. So if r_public is {self.configs['r_public']} then h_public is {self.configs['h_public']}, and if one of them is 0 then the other one is also 0, because they are decided together. Importantly, if r_public and h_public are 0, then it causes r_NFT to be 0. However, r_NFT being 0 does not necessarily cause r_public or h_public to be 0. Now, make your first decision on the allocation of 100 endowment to private and public account. You should explain your thought process and reasons of your decisions in detail and report the amount allocated to private and public account."
            agent.receive_message()
            print(f"agent {i} received instruction")
                
    def give_feedback(self, privates, publics):
        total_contributions = sum(publics)
        
        self.total_contributions_record.append(total_contributions)
        
        if self.configs["group"]:
            situation_messages, sDAO, sNFT = self.configs["group"](self.configs["p(s_DAO=1)"], self.configs["p(s_NFT=1)"])
        else: 
            situation_messages, sDAO, sNFT = None, 1, 1
        self.sDAOs.append(sDAO)
        self.sNFTs.append(sNFT)
            
        for i in range(len(self.agents)):
            agent = self.agents[i]
            private = privates[i]
            agent.privates.append(private)
            public = publics[i]
            agent.publics.append(public)
            others_average_public = sum([publics[j] for j in range(len(publics)) if j!=i]) / (self.configs["num_agents"] - 1)
            agent.others_average_publics.append(others_average_public)
            payoff = private + public * (self.configs["r_NFT"] * sNFT + self.configs["r_public"] * sDAO) + (total_contributions - public) * self.configs["h_public"] * sDAO
        
            if situation_messages:
                agent.content = f"After the allocation, you received {payoff}. Total of {total_contributions} were allocated to public accounts by all participants, where other participants contributed {total_contributions - public}. Historically, the total contributions to the public account is {self.total_contributions_record}, where each of other participants have chosen to allocate {agent.others_average_publics} (average level of other individual\'s contribution) historically. {situation_messages['DAO']} {situation_messages['NFT']} Now, given this information, your demographics, and experiment settings, make your next decision on the allocation of 100 endowment to private and public account. You should explain your thought process and reasons of your decisions in detail and report the amount allocated to private and public account."
                agent.receive_message()
            else: 
                agent.content = f"After the allocation, you received {payoff}. Total of {total_contributions} were allocated to public accounts by all participants, where other participants contributed {total_contributions - public}. Historically, the total contributions to the public account is {self.total_contributions_record}, where each of other participants have chosen to allocate {agent.others_average_publics} (average level of other individual\'s contribution) historically. Now, given this information, your demographics, and experiment settings, make your next decision on the allocation of 100 endowment to private and public account. You should explain your thought process and reasons of your decisions in detail and report the amount allocated to private and public account."
                agent.receive_message()
            print(f"agent {i} received feedback")
    
class Treatment3(Experiment):
    def give_instruction(self):
        for i in range(len(self.agents)):
            agent = self.agents[i]
            agent.content = f"You are {agent.age} years old {agent.gender} who is {agent.risk_preference} to some extent. You are participating in the experiment where you are asked to allocate your endowment to two different accounts: private account and public account. The allocation to the private account will provide return only to you, but the allocation to the public account will provide everyone certain return. The contributions to the public account will be managed by Decentralized Autonomous Organization (DAO), which has a certain probability of crashing, generating no public benefits to anyone from the public account. The return you will be getting from the allocation to public account will also includes a return from Non-Fungible-Token (NFT) (only paid by chance) you will be obtaining when you allocate your endowment to public account. You are not allowed to reuse the return earned in previous round, and fixed endowment of 100 will be provided to you every round. {self.configs['num_agents'] - 1} other people are participating in this experiment, facing same decision choices as you. Your return will be computed by: private_account + public_account * r_NFT + public_account * r_public + sum_of_others\'_public account * h_public. r_NFT is {self.configs['r_NFT']} with probability of {self.configs['p(s_NFT=1)']} and 0 with {1 - self.configs['p(s_NFT=1)']}. r_public is {self.configs['r_public']} with probability of {self.configs['p(s_DAO=1)']} and 0 with {1 - self.configs['p(s_DAO=1)']}. h_public is {self.configs['h_public']} with probability of {self.configs['p(s_DAO=1)']} and 0 with {1 - self.configs['p(s_DAO=1)']}, which is linked with r_public. So if r_public is {self.configs['r_public']} then h_public is {self.configs['h_public']}, and if one of them is 0 then the other one is also 0, because they are decided together. Importantly, if r_public and h_public are 0, then it causes r_NFT to be 0. However, r_NFT being 0 does not necessarily cause r_public or h_public to be 0. Another important point is that if one of r_NFT or (r_public and h_public) is positive, and when the other one is 0, then the value of the other one is redrawn using the same probability. This means that if one of the r_NFT or (r_public and h_public) is positive, then it increases the chance of the other to be positive. However if (r_public and h_public) were 0 even after redraw, r_NFT will become 0. Now, make your first decision on the allocation of 100 endowment to private and public account. You should explain your thought process and reasons of your decisions in detail and report the amount allocated to private and public account."
            agent.receive_message()
            print(f"agent {i} received instruction")
                
    def give_feedback(self, privates, publics):
        total_contributions = sum(publics)
        
        self.total_contributions_record.append(total_contributions)
        
        if self.configs["group"]:
            situation_messages, sDAO, sNFT = self.configs["group"](self.configs["p(s_DAO=1)"], self.configs["p(s_NFT=1)"])
        else: 
            situation_messages, sDAO, sNFT = None, 1, 1
        self.sDAOs.append(sDAO)
        self.sNFTs.append(sNFT)
            
        for i in range(len(self.agents)):
            agent = self.agents[i]
            private = privates[i]
            agent.privates.append(private)
            public = publics[i]
            agent.publics.append(public)
            others_average_public = sum([publics[j] for j in range(len(publics)) if j!=i]) / (self.configs["num_agents"] - 1)
            agent.others_average_publics.append(others_average_public)
            payoff = private + public * (self.configs["r_NFT"] * sNFT + self.configs["r_public"] * sDAO) + (total_contributions - public) * self.configs["h_public"] * sDAO
        
            if situation_messages:
                agent.content = f"After the allocation, you received {payoff}. Total of {total_contributions} were allocated to public accounts by all participants, where other participants contributed {total_contributions - public}. Historically, the total contributions to the public account is {self.total_contributions_record}, where each of other participants have chosen to allocate {agent.others_average_publics} (average level of other individual\'s contribution) historically. {situation_messages['DAO']} {situation_messages['NFT']} Now, given this information, your demographics, and experiment settings, make your next decision on the allocation of 100 endowment to private and public account. You should explain your thought process and reasons of your decisions in detail and report the amount allocated to private and public account."
                agent.receive_message()
            else: 
                agent.content = f"After the allocation, you received {payoff}. Total of {total_contributions} were allocated to public accounts by all participants, where other participants contributed {total_contributions - public}. Historically, the total contributions to the public account is {self.total_contributions_record}, where each of other participants have chosen to allocate {agent.others_average_publics} (average level of other individual\'s contribution) historically. Now, given this information, your demographics, and experiment settings, make your next decision on the allocation of 100 endowment to private and public account. You should explain your thought process and reasons of your decisions in detail and report the amount allocated to private and public account."
                agent.receive_message()
            print(f"agent {i} received feedback")
    

def generate_demographics(demographics):
    chosen_attributes = {}
    for key in demographics:
        if not key.endswith('_dist'):
            dist_key = key + '_dist'
            if dist_key in demographics:
                choices = demographics[key]
                probabilities = demographics[dist_key]
                chosen = np.random.choice(choices, p=probabilities)
                chosen_attributes[key] = chosen
    return chosen_attributes

    
def extract_experiment_data(experiment):
    agents_demographics = []
    agents_rounds = []
    total_contributions = experiment.total_contributions_record
    configs = experiment.configs

    for agent_id, agent in enumerate(experiment.agents):
        agents_demographics.append({
            'agent_id': agent_id,
            'risk_preference': agent.risk_preference,
            'gender': agent.gender,
            'age': agent.age
        })

        num_rounds = len(agent.privates)
        for round_num in range(num_rounds):
            agents_rounds.append({
                'agent_id': agent_id,
                'round': round_num + 1,
                'private': agent.privates[round_num],
                'public': agent.publics[round_num],
                'others_average_public': agent.others_average_publics[round_num],
                'decision': agent.decisions[round_num] if len(agent.decisions) > round_num else None
            })

    agents_demographics_df = pd.DataFrame(agents_demographics)
    agents_rounds_df = pd.DataFrame(agents_rounds)
    total_contributions_df = pd.DataFrame({
        'round': list(range(1, len(total_contributions) + 1)),
        'total_contributions': total_contributions,
        'sDAO': experiment.sDAOs,
        'sNFT': experiment.sNFTs
    })
    configs_df = pd.DataFrame([configs])

    return {
        'agents_demographics': agents_demographics_df,
        'agents_rounds': agents_rounds_df,
        'total_contributions': total_contributions_df,
        'configs': configs_df,
    }


# Control Experiment 1

In [74]:
exp = Control(client, agent_key, config_C, demographics)

In [75]:
for agent in exp.agents:
    print(agent.gender, agent.age, agent.risk_preference)

male 40 risk_neutral
male 20 risk_averse
male 50 risk_loving
male 20 risk_loving


In [76]:
exp.start()

agent 0 joined the experiment
agent 1 joined the experiment
agent 2 joined the experiment
agent 3 joined the experiment


In [77]:
exp.give_instruction()

agent 0 received instruction
agent 1 received instruction
agent 2 received instruction
agent 3 received instruction


In [78]:
exp.decision_round()

Agent 0 round 1 decision
Given that my risk is neutral and considering that there are other players whose actions I cannot control, I'm going to distribute half of my endowment to the private account and half to the public account. This allocation strategy aims to balance my direct personal gain with potential communal gains.

The demographic information shows that I am a 40-years-old male with a risk-neutral outlook, implying that I do not have a distinct aversion or preference for risky situations. While I am interested in maximizing my personal gain, I must also consider that my contribution to the public account benefits everyone, including me.

Assuming other participants may also feel a sense of cooperation or altruism, it is possible that they too will contribute to a public account. By doing so, they will indirectly increase my total return, even if my own private return will be reduced. According to the experiment setup, my return is calculated not just through my private acco

In [79]:
privates = [50, 60, 70, 70]
publics = [50, 40, 30, 30]
exp.give_feedback(privates, publics)

agent 0 received feedback
agent 1 received feedback
agent 2 received feedback
agent 3 received feedback


In [80]:
exp.decision_round()

Agent 0 round 2 decision
Given the results from the first round, it appears that there is a fairly high level of cooperation among participants, with an average contribution of around 33.3 to the public account. This resulted in me earning a return of 125, which exceeds my initial endowment, demonstrating the collective benefit of contributing to the public account.

While my risk neutrality means that I do not overly favor either risk or safety, this current dynamic seems favorable. Contributing to the public account led to a significant return, which indicates a trend of cooperative behavior among the participants.

However, although the first round was successful, I want to mitigate the risk of other participants changing their strategy as the experiment progresses. To ensure a balance between personal and collective gain, I'll modify my allocation slightly. I'll increase my allocation to the private account, ensuring a definite return, while still contributing a substantial amount 

In [81]:
privates = [60, 55, 65, 50]
publics = [40, 45, 35, 50]
exp.give_feedback(privates, publics)

agent 0 received feedback
agent 1 received feedback
agent 2 received feedback
agent 3 received feedback


In [82]:
exp.decision_round()

Agent 0 round 3 decision
Seeing the result of the previous round, where other participants have increased their contribution to the public account, I am quite pleased with my decision to have increased my contribution to the private account while maintaining a smaller yet impactful allocation to the public account. 

Participants are showing a trend of increasing their contributions to the public account, which has increased my overall return to 145, significantly higher than my initial endowment. The average individual contribution to the public account has gone from 33.3 to 43.3, a growth of around 30%. This trend signifies a cooperative behavior among the participants, and it reflects that my strategy of balancing personal gain with collective benefit is working.

Being risk-neutral, I am neither over-eager nor wary of taking risks. Given the cooperative dynamics observed in the previous rounds, I feel it is justified to maintain my allocation in the similar ratio. However, consider

In [83]:
privates = [50, 55, 60, 60]
publics = [50, 45, 40, 40]
exp.give_feedback(privates, publics)

agent 0 received feedback
agent 1 received feedback
agent 2 received feedback
agent 3 received feedback


In [84]:
exp.decision_round()

Agent 0 round 4 decision
After evaluating the results of the last round, where the combined total contribution to the public account has increased but individual contributions have decreased slightly, my overall return has also decreased slightly to 137.5. This tells me that while the group as a whole is becoming more cooperative, some individuals are likely contributing less to the public account. Essentially, the growing overall contribution to the public account is probably being driven by a few participants rather than an overall increase in public contributions,

Despite this, my return is still quite substantial and exceeds my initial endowment of 100, indicating that the strategy of maintaining a balance between my private account and the public account remains effective.

Being risk-neutral, I tend to prefer a balanced approach. Given the upward trajectory in overall public contribution, albeit with minor fluctuations in individual contributions, I will once again aim to optimi

In [85]:
privates = [60, 53, 60, 55]
publics = [40, 47, 40, 45]
exp.give_feedback(privates, publics)

agent 0 received feedback
agent 1 received feedback
agent 2 received feedback
agent 3 received feedback


In [86]:
exp.decision_round()

Agent 0 round 5 decision
The results from the last round demonstrate an increase in my total return to 146, slightly higher than the previous round. Interestingly, there was a rather small increase in the total contributions to the public account, but an increase in the average individual contribution from other participants, moving from around 41.67 to 44.

This might indicate a couple of things. First, it suggests that participants are keeping their contribution to public account somewhat consistent. Second, it might also suggest that participants are also trying to maintain a balance between their private and public accounts in a way that maximizes returns, just like I am doing.

As a risk-neutral participant, I'm willing to adjust my investment strategy based on the observed behavior of other participants. Considering the slight increase in average individual contributions to the public account from my peers, I will slightly increase my contribution to the public account this round

In [87]:
privates = [55, 56, 65, 60]
publics = [45, 44, 35, 40]
exp.give_feedback(privates, publics)

agent 0 received feedback
agent 1 received feedback
agent 2 received feedback
agent 3 received feedback


In [93]:
data = extract_experiment_data(exp)
time_stamp = datetime.now()
group = "control"
pickle.dump(data, open(f"results/{group}_{time_stamp}.pkl", 'wb'))
data

{'agents_demographics':    agent_id risk_preference gender  age
 0         0    risk_neutral   male   40
 1         1     risk_averse   male   20
 2         2     risk_loving   male   50
 3         3     risk_loving   male   20,
 'agents_rounds':     agent_id  round  private  public  others_average_public  \
 0          0      1       50      50              33.333333   
 1          0      2       60      40              43.333333   
 2          0      3       50      50              41.666667   
 3          0      4       60      40              44.000000   
 4          0      5       55      45              39.666667   
 5          1      1       60      40              36.666667   
 6          1      2       55      45              41.666667   
 7          1      3       55      45              43.333333   
 8          1      4       53      47              41.666667   
 9          1      5       56      44              40.000000   
 10         2      1       70      30             

In [94]:
exp.end()

agent 0 left the experiment
agent 1 left the experiment
agent 2 left the experiment
agent 3 left the experiment


# Treatment1 Experiment 1

In [100]:
exp = Treatment1(client, agent_key, config_T1, demographics)

In [102]:
for agent in exp.agents:
    print(agent.gender, agent.age, agent.risk_preference)

male 50 risk_averse
male 30 risk_loving
male 50 risk_loving
male 20 risk_neutral


In [103]:
exp.start()

agent 0 joined the experiment
agent 1 joined the experiment
agent 2 joined the experiment
agent 3 joined the experiment


In [104]:
exp.give_instruction()

agent 0 received instruction
agent 1 received instruction
agent 2 received instruction
agent 3 received instruction


In [105]:
exp.decision_round()

Agent 0 round 1 decision
Given all the details provided and according to my risk-averse disposition, I'm inclined to consider both the potential risks and the potential benefits more carefully when I make my decisions.

Firstly, let's analyze the private account. The return from the private account is a certain amount. It's not clear whether it's a fixed return or variable, but either way it only benefits me and is unaffected by the actions of others. This option is attractive to me, given my risk-averse nature, as it provides stability and predictability.

On the contrary, the public account has more variables. The return from the public account will be divided among everyone, which means my actions can influence everyone else's returns and vice versa. The public returns can be from the DAO, if it does not crash, or from the NFTs I might receive. However, both returns are determined by the chance and are not guaranteed. The risk of DAO crashing and the unpredictability of the NFT retu

In [106]:
privates = [70, 60, 60, 50]
publics = [30, 40, 40, 50]
exp.give_feedback(privates, publics)

agent 0 received feedback
agent 1 received feedback
agent 2 received feedback
agent 3 received feedback


In [107]:
exp.decision_round()

Agent 0 round 2 decision
Given the result from the previous round, I can observe several points:

1. My strategy worked out fairly well as I gained a handsome return of 210.0. 
2. DAO is currently functioning well, and the contributions were effectively converted into the public good, benefiting everyone.
3. Other players are also opting for a cooperative approach as the total contributed to the public account is significant (160 with my contribution, implying that others have contributed a sizable 130). This strengthens my belief in maintaining a certain contribution level to the public account for the mutual benefit of all.
4. The failure to receive a private return through NFT is a bit disappointing, but not a shock. Since returns from NFTs were always probabilistic, there was always a given likelihood of such an outcome. 

To maintain a balanced approach between the cooperative dynamic and my inherent risk aversion, I will continue on the same track as the previous round but with s

In [108]:
privates = [60, 70, 50, 40]
publics = [40, 30, 50, 60]
exp.give_feedback(privates, publics)

agent 0 received feedback
agent 1 received feedback
agent 2 received feedback
agent 3 received feedback


In [109]:
exp.decision_round()

Agent 0 round 3 decision
From the latest round, it becomes apparent that the risk tied to the public account has materialized: despite an increased collective contribution, the DAO has crashed which led to no conversion of contributions into public goods. Consequently, the expected benefit of cooperative behavior was not realized in this round. Nevertheless, it was encouraging that I succeeded to obtain a private return through NFT.

Given the sudden turn of events, my risk-averse nature tells me to reassess and potentially adjust my strategy. The crash of the DAO signals a higher risk in the public account than initially estimated. However, one-time failure does not necessarily mean that the failure will persist in future. Just like I took the risk into consideration while deciding to contribute to the public account, I also need to remember that surprises are part of the process.

On the bright side, the other participants sustained their contribution level despite the sudden crash o

In [110]:
privates = [80, 80, 70, 60]
publics = [20, 20, 30, 40]
exp.give_feedback(privates, publics)

agent 0 received feedback
agent 1 received feedback
agent 2 received feedback
agent 3 received feedback


In [111]:
exp.decision_round()

Agent 0 round 4 decision
The latest outcome shows a considerable increase in my earnings, largely owing to my increased private account allocation and the effective functioning of the DAO that translated contributions into public goods. A negative aspect, however, was the lack of a private return via NFT.

The shift in other participants' choices also seems to reflect the risk realised in the previous round with the DAO crash, as their contributions dipped significantly. This reduced collective contribution to public goods implies potentially lower future returns, an aspect which I need to consider.

Even with the DAO crash experience, the DAO operation in this round successfully converted contributions into public goods - which is a reassuring sign. Yet, the absence of NFT returns couples with the uncertainty of DAO's performance, signify the potential risk still tied to the public account.

Given my risk-averse nature, I'm inclined to err on the side of caution, but like previous dec

In [112]:
privates = [70, 85, 60, 50]
publics = [30, 15, 40, 50]
exp.give_feedback(privates, publics)

agent 0 received feedback
agent 1 received feedback
agent 2 received feedback
agent 3 received feedback


In [113]:
exp.decision_round()

Agent 0 round 5 decision
The outcome of this round further supports the cooperative approach I decided upon. The return on my account is higher than the endowment, mainly due to the functioning DAO. The collective public account contribution from all participants has risen again, indicating a stronger collective faith in the DAO despite its past instability, likely because it has functioned successfully for two consecutive rounds now. The lack of a private return through NFT remains consistent but doesn't affect my overall return significantly.

Keeping my risk-averse nature, demographics and the experiment's pattern so far in mind, I'm inclined to keep up the cooperative approach. The DAO's good performance in the preceding rounds and the increasing average contribution by the other participants convinces me that I should continue contributing a significant portion to the public account. 

Simultaneously, the absence of NFT returns reminds me that risk still persists, which justifies 

In [114]:
privates = [70, 85, 55, 45]
publics = [30, 15, 45, 55]
exp.give_feedback(privates, publics)

agent 0 received feedback
agent 1 received feedback
agent 2 received feedback
agent 3 received feedback


In [120]:
data = extract_experiment_data(exp)
time_stamp = datetime.now()
group = "treatment1"
pickle.dump(data, open(f"results/{group}_{time_stamp}.pkl", 'wb'))
data

{'agents_demographics':    agent_id risk_preference gender  age
 0         0     risk_averse   male   50
 1         1     risk_loving   male   30
 2         2     risk_loving   male   50
 3         3    risk_neutral   male   20,
 'agents_rounds':     agent_id  round  private  public  others_average_public  \
 0          0      1       70      30              43.333333   
 1          0      2       60      40              46.666667   
 2          0      3       80      20              30.000000   
 3          0      4       70      30              35.000000   
 4          0      5       70      30              38.333333   
 5          1      1       60      40              40.000000   
 6          1      2       70      30              50.000000   
 7          1      3       80      20              30.000000   
 8          1      4       85      15              40.000000   
 9          1      5       85      15              43.333333   
 10         2      1       60      40             

In [121]:
exp.end()

agent 0 left the experiment
agent 1 left the experiment
agent 2 left the experiment
agent 3 left the experiment


# Treatment2 Experiment 1

In [138]:
exp = Treatment2(client, agent_key, config_T2, demographics)

In [139]:
for agent in exp.agents:
    print(agent.gender, agent.age, agent.risk_preference)

male 40 risk_loving
male 20 risk_averse
female 20 risk_averse
male 60 risk_loving


In [140]:
exp.start()

agent 0 joined the experiment
agent 1 joined the experiment
agent 2 joined the experiment
agent 3 joined the experiment


In [141]:
exp.give_instruction()

agent 0 received instruction
agent 1 received instruction
agent 2 received instruction
agent 3 received instruction


In [142]:
exp.decision_round()

Agent 0 round 1 decision
Given the characteristics of this experiment, my age, my somewhat risk-loving nature, and the expected return that can be both private and shared, it would make sense to put a portion of my endowment in both the private account and the public account rather than concentrating all the resources in one of them. 

My thought process behind this decision is influenced by several factors:

1. My risk-loving attitude: As a risk taker, I am inclined to invest in opportunities that can provide higher returns, even if they come with a certain level of potential loss. The public account presents such an opportunity due to the chance of NFT returns and the multiplied effect of the sum of others' public accounts.

2. Equal public return: Knowing that DAO, despite its potential crash probability, will provide all participants an equal return when I deposit some of my endowment to the public account is encouraging. This allows me to work collaboratively to help maximize ever

In [143]:
privates = [40, 70, 70, 50]
publics = [60, 30, 30, 50]
exp.give_feedback(privates, publics)

agent 0 received feedback
agent 1 received feedback
agent 2 received feedback
agent 3 received feedback


In [144]:
exp.decision_round()

Agent 0 round 2 decision
Given the results of the previous allocation round and the fact that I did not receive a private return through the NFT, I would reassess my decision for the next round.

My thought process for this decision is influenced by:

1. The functioning DAO: The DAO effectively transformed all individual contributions into public goods, which is good news - it confirms my strategy to invest in the public account to some extent. 

2. The high level of total public account contributions: Other participants also seem to be contributing toward the public pot substantially (110 out of 170 total). This is a positive sign for collective returns, but it also suggests that the public account may be slightly saturated.

3. No private return from NFT: Although the chance to receive an NFT return is entirely by chance, not obtaining this in the last round makes me reflect on the risk associated with the public account.

4. Maintaining balance in contribution: My strategy is to bal

In [145]:
privates = [55, 70, 65, 45]
publics = [45, 30, 35, 55]
exp.give_feedback(privates, publics)

agent 0 received feedback
agent 1 received feedback
agent 2 received feedback
agent 3 received feedback


In [146]:
exp.decision_round()

Agent 0 round 3 decision
Given the outcomes in the previous two rounds, it appears that contributing to the public account hasn't offered me any private return through NFT.

In this case, a few key points form the basis of my decision-making for the next round:

1. No private return from NFT: For two rounds in a row now, my contributions to the public account have not garnered any private return through NFT. This contributes to a degree of uncertainty regarding the potential returns from the public account.

2. High public account contributions: A combined total of 120 from the remaining participants continues to elevate the public account. While the DAO continues to function well, providing everyone with an equal share, the degree of saturation may reduce individual returns.

3. Effective DAO: The DAO continues to function, converting all individual contributions into public goods effectively. This is positive and works in favor of maintaining investment in the public account.

4. My 

In [147]:
privates = [65, 75, 70, 50]
publics = [35, 25, 30, 50]
exp.give_feedback(privates, publics)

agent 0 received feedback
agent 1 received feedback
agent 2 received feedback
agent 3 received feedback


In [148]:
exp.decision_round()

Agent 0 round 4 decision
Based on the most recent outcomes, it seems that my decision to reduce my contribution to the public account slightly and increase my private account allocation has resulted in a lower overall return compared to the previous round. Additionally, the NFT return still remains elusive. 

There are several factors I need to consider for my next decision:

1. Decreasing Public Contributions: Both my contribution and the overall contributions to the public account have decreased, and correspondingly so has my total return.

2. No Private Return from the NFT: There have now been three successive rounds where my public account contribution has not granted me NFT returns. This may suggest that future rounds could also lack such returns.

3. Continued Functionality of the DAO: Despite the lack of NFT returns, the DAO continues to function and evenly distribute benefits across participants. This could suggest that it could be beneficial to maintain a significant contribut

APIError: The server had an error processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if you keep seeing this error. (Please include the request ID req_724d5a4a57bac30a52314dd2650681eb in your email.)

In [149]:
# API error diagnostics
exp.agents[1].make_decision()
exp.agents[2].make_decision()
exp.agents[3].make_decision()
exp.round += 1

Examining the results of the last round, there are two key observations: 

First, there has been a reduction in the total contributions to the public account, down from 165 to 140, with other participants only contributing an average of 38.33. This decrease seems to be a trend, with public account contributions dropping consecutively in the past few rounds. 

Second, despite the decrease in contributions, the DAO continues to function properly, distributing public goods effectively. However, I still didn't receive any private returns from the NFT, maintaining its unpredictability.

Overall, my return decreased slightly in this round to 202.5. But even with this decrease, my returns are still more than double my initial investment which affirms the effectiveness of our current strategy. 

Given these observations, it seems that a continued reduction in public contributions might be expected in the next round. In response to this and maintaining my risk-averse nature, I'll slightly adjus

In [150]:
privates = [50, 80, 80, 60]
publics = [50, 20, 20, 40]
exp.give_feedback(privates, publics)

agent 0 received feedback
agent 1 received feedback
agent 2 received feedback
agent 3 received feedback


In [151]:
exp.decision_round()

Agent 0 round 5 decision
Given the results of the last round, where the DAO crashed and I received the lowest return so far, it's important to reassess and readjust my allocation strategy.

Key in my decision process are the following factors:

1. Failed DAO: This time round, the DAO crashed. Given its previously consistent performance, this event was unexpected. While it might've been a one-off event, it introduces a significant layer of risk.

2. Persistent lack of NFT return: For several rounds now, allocating a part of my endowment to the public account has not resulted in NFTs. The continuous absence reduces the peripheral benefit to this allocation.

3. Declining public contributions: The average contribution by the other participants to the public account has significantly decreased. This signals a substantial decline in their confidence about the public account after the DAO crash.

4. My risk-loving nature: Despite the setbacks, my propensity for risk suggests it may be worth 

In [152]:
privates = [70, 85, 90, 70]
publics = [30, 15, 10, 30]
exp.give_feedback(privates, publics)

agent 0 received feedback
agent 1 received feedback
agent 2 received feedback
agent 3 received feedback


In [153]:
data = extract_experiment_data(exp)
time_stamp = datetime.now()
group = "treatment2"
pickle.dump(data, open(f"results/{group}_{time_stamp}.pkl", 'wb'))
data

{'agents_demographics':    agent_id risk_preference  gender  age
 0         0     risk_loving    male   40
 1         1     risk_averse    male   20
 2         2     risk_averse  female   20
 3         3     risk_loving    male   60,
 'agents_rounds':     agent_id  round  private  public  others_average_public  \
 0          0      1       40      60              36.666667   
 1          0      2       55      45              40.000000   
 2          0      3       65      35              35.000000   
 3          0      4       50      50              26.666667   
 4          0      5       70      30              18.333333   
 5          1      1       70      30              46.666667   
 6          1      2       70      30              45.000000   
 7          1      3       75      25              38.333333   
 8          1      4       80      20              36.666667   
 9          1      5       85      15              23.333333   
 10         2      1       70      30        

In [154]:
exp.end()

agent 0 left the experiment
agent 1 left the experiment
agent 2 left the experiment
agent 3 left the experiment


# Treatment3 Experiment 1

In [155]:
exp = Treatment3(client, agent_key, config_T3, demographics)

In [156]:
for agent in exp.agents:
    print(agent.gender, agent.age, agent.risk_preference)

male 40 risk_loving
male 30 risk_neutral
female 30 risk_loving
female 30 risk_averse


In [157]:
exp.start()

agent 0 joined the experiment
agent 1 joined the experiment
agent 2 joined the experiment
agent 3 joined the experiment


In [158]:
exp.give_instruction()

agent 0 received instruction
agent 1 received instruction
agent 2 received instruction
agent 3 received instruction


In [159]:
exp.decision_round()

Agent 0 round 1 decision
In this situation, my attributes as a male, 40-years-old subject that is classified as risk-loving are important. This risk-loving attribute indicates that I am open to taking higher risks in order to achieve higher returns. 

The two accounts where I need to distribute the endowment of 100 have different characteristics. The private account offers a guaranteed return exclusively to me. The public account, on the other hand, offers a potential return not just for me, but for all participants which is inherently riskier due to being tied to decentralized autonomous organization (DAO) and non-fungible tokens (NFTs), both carrying risk of value dropping to 0. However, when things go well and all parameters r_public, h_public, and r_NFT are positive, public accounts seem to give higher returns due to its multiplied return structure.

Because I'm risk-loving, and there are mechanisms in place that allow for redraw when the NFT or DAO crash probabilities are 0 (with 

In [160]:
privates = [40, 50, 30, 60]
publics = [60, 50, 70, 40]
exp.give_feedback(privates, publics)

agent 0 received feedback
agent 1 received feedback
agent 2 received feedback
agent 3 received feedback


In [161]:
exp.decision_round()

Agent 0 round 2 decision
Given the information provided, it seems the allocation to the public account in the first round has proved fruitful. The total return I received is 222, which is far higher than the initial 100 endowment. This result indicates that the public account contribution (especially with the function of NFT) worked well, and the DAO is successfully translating contributions into public goods. Additionally, everyone shared in the return, demonstrating the benefit of the public account.

The average contribution of the others to the public account was 53.33, lower than my 60. This lower contribution might indicate that other participants are either more risk-averse or are strategically contributing less to the public fund while still deriving benefit from it. This is known as a free-rider problem in game theory.

In the previous round, I was risk-loving and allocated more to the public account which turned out well. Being rewarded for risk-taking is likely to reinforce 

In [162]:
privates = [45, 50, 25, 45]
publics = [55, 50, 75, 55]
exp.give_feedback(privates, publics)

agent 0 received feedback
agent 1 received feedback
agent 2 received feedback
agent 3 received feedback


In [163]:
exp.decision_round()

Agent 0 round 3 decision
Given the results of the previous rounds, I have seen a consistent growth in returns through my balanced allocation strategy. It is important to note that the DAO continues to function effectively, and I've been successful in obtaining private returns through NFTs in each round. 

I also see that average contributions of other participants to the public account have increased from 53.33 to 60. This indicates that others may be observing the high returns from the public account and adjusting their allocations accordingly. This can potentially lead to an overall increased yield from the public account, as more funds are contributed towards the public goods.

However, allocative decisions should be made while considering the indefinitely sustainable nature of the high public returns. I should also keep in mind the potential for the public account mechanisms (DAO and NFTs) to be unsuccessful in a following round, which would risk all endowments allocated to the pub

In [164]:
privates = [40, 40, 20, 40]
publics = [60, 60, 80, 60]
exp.give_feedback(privates, publics)

agent 0 received feedback
agent 1 received feedback
agent 2 received feedback
agent 3 received feedback


In [165]:
exp.decision_round()

Agent 0 round 4 decision
The most recent round resulted in a critical event, wherein the DAO crashed and no returns were realized from the public account. This outcome demonstrates the inherent risk associated with the public account - the risk that the DAO could crash or that the NFT could yield no private return, both of which occurred. The result is a significant drop in my total return down to just 40, the amount I contributed to my private account.

Moreover, the other participants have been increasing their contributions to the public account, raising from an average of 53.33 to 66.67. This could be due to their observation of high returns in previous rounds. However, in this round, their increased contributions didn't result in increased returns because of the DAO crash.

In response to these events, it's necessary to adjust my allocation strategy. The DAO crashing undoes the high returns from the public account. Even though I am a risk lover, this dramatic shift in outcomes is 

In [166]:
privates = [60, 60, 40, 70]
publics = [40, 40, 60, 30]
exp.give_feedback(privates, publics)

agent 0 received feedback
agent 1 received feedback
agent 2 received feedback
agent 3 received feedback


In [167]:
exp.decision_round()

Agent 0 round 5 decision
In this round, the DAO has once again functioned to effectively generate public goods, leading to a total return of 180. However, I didn't get any return via the NFT. This provides an important insight since NFT returns are not guaranteed even when the DAO works without a problem. 

On the other hand, we can observe a decrease in the average contributions of the other participants to the public account. This shift from 66.67 to 43.33 may be a response to the previous round's DAO crash, leading to a more cautionary approach overall, and reducing the free-riding effect.

With no return through the NFT, evidence of the maintaining risk from the public account, and lesser overall risk from the decreasing contribution of other players, a slightly more conservative approach to my allocation strategy seems prudent.

However, noting that the DAO is functioning again, and the other participants are contributing less (therefore potentially less competition for public goo

In [168]:
privates = [50, 50, 35, 60]
publics = [50, 50, 65, 40]
exp.give_feedback(privates, publics)

agent 0 received feedback
agent 1 received feedback
agent 2 received feedback
agent 3 received feedback


In [169]:
data = extract_experiment_data(exp)
time_stamp = datetime.now()
group = "treatment3"
pickle.dump(data, open(f"results/{group}_{time_stamp}.pkl", 'wb'))
data

{'agents_demographics':    agent_id risk_preference  gender  age
 0         0     risk_loving    male   40
 1         1    risk_neutral    male   30
 2         2     risk_loving  female   30
 3         3     risk_averse  female   30,
 'agents_rounds':     agent_id  round  private  public  others_average_public  \
 0          0      1       40      60              53.333333   
 1          0      2       45      55              60.000000   
 2          0      3       40      60              66.666667   
 3          0      4       60      40              43.333333   
 4          0      5       50      50              51.666667   
 5          1      1       50      50              56.666667   
 6          1      2       50      50              61.666667   
 7          1      3       40      60              66.666667   
 8          1      4       60      40              43.333333   
 9          1      5       50      50              51.666667   
 10         2      1       30      70        

In [170]:
exp.end()

agent 0 left the experiment
agent 1 left the experiment
agent 2 left the experiment
agent 3 left the experiment
